In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
TRAIN_IMAGE_DIR = '../input/train/images/'
TRAIN_MASK_DIR = '../input/train/masks/'
TEST_IMAGE_DIR = '../input/test/images/'

In [ ]:
train_fns = os.listdir(TRAIN_IMAGE_DIR)

## U-net approach

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D
from keras.layers import BatchNormalization
from keras.layers import Dropout, MaxPooling2D
from keras.layers import Activation, concatenate
from keras.layers import Conv2DTranspose

from keras.optimizers import Adam
from keras import losses

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [ ]:
X = [np.array(cv2.resize(cv2.imread(TRAIN_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), (128, 128)), 
              dtype=np.uint8) for p in tqdm(train_fns)]
X = np.array(X)/255
X = np.expand_dims(X,axis=3)

In [ ]:
y = [np.array(cv2.resize(cv2.imread(TRAIN_MASK_DIR + p, cv2.IMREAD_GRAYSCALE), 
                         (128, 128)), dtype = np.uint8) for p in tqdm(train_fns)]
y = np.array(y) / 255
y = np.expand_dims(y, axis = 3)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [ ]:
def conv_block(neurons, block_input, dropout=None):
    conv1 = Conv2D(neurons, (3,3), padding='same')(block_input)
    conv1 = Activation('relu')(conv1)
    conv1 = BatchNormalization()(conv1)
    if dropout is not None:
        conv1 = Dropout(dropout)(conv1)
    conv2 = Conv2D(neurons, (3,3), padding='same')(conv1)
    conv2 = Activation('relu')(conv2)
    conv2 = BatchNormalization()(conv2)
    if dropout is not None:
        conv2 = Dropout(dropout)(conv2)
    pool = MaxPooling2D((2,2))(conv2)
    return pool, conv2  # returns the block output and the shortcut to use in the uppooling blocks

def middle_block(neurons, block_input, dropout=None):
    conv1 = Conv2D(neurons, (3,3), padding='same')(block_input)
    conv1 = Activation('relu')(conv1)
    conv1 = BatchNormalization()(conv1)
    if dropout is not None:
        conv1 = Dropout(dropout)(conv1)
    conv2 = Conv2D(neurons, (3,3), padding='same')(conv1)
    conv2 = Activation('relu')(conv2)
    conv2 = BatchNormalization()(conv2)
    if dropout is not None:
        conv2 = Dropout(dropout)(conv2)
    
    return conv2

def deconv_block(neurons, block_input, shortcut, dropout=None):
    deconv = Conv2DTranspose(neurons, (3, 3), strides=(2, 2), padding="same")(block_input)
    uconv = concatenate([deconv, shortcut])
    uconv = Conv2D(neurons, (3, 3), padding="same")(uconv)
    uconv = Activation('relu')(uconv)
    uconv = BatchNormalization()(uconv)
    if dropout is not None:
        uconv = Dropout(dropout)(uconv)
    uconv = Conv2D(neurons, (3, 3), padding="same")(uconv)
    uconv = Activation('relu')(uconv)
    uconv = BatchNormalization()(uconv)
    if dropout is not None:
        uconv = Dropout(dropout)(uconv)
        
    return uconv
    
def build_model(start_neurons=8, dropout=None):
    
    input_layer = Input((128, 128, 1))
    
    # 128 -> 64
    conv1, shortcut1 = conv_block(start_neurons, input_layer, dropout)

    # 64 -> 32
    conv2, shortcut2 = conv_block(start_neurons * 2, conv1, dropout)
    
    # 32 -> 16
    conv3, shortcut3 = conv_block(start_neurons * 4, conv2, dropout)
    
    # 16 -> 8
    conv4, shortcut4 = conv_block(start_neurons * 8, conv3, dropout)
    
    # Middle
    convm = middle_block(start_neurons * 16, conv4, dropout)
    
    # 8 -> 16
    deconv4 = deconv_block(start_neurons * 8, convm, shortcut4, dropout)
    
    # 16 -> 32
    deconv3 = deconv_block(start_neurons * 4, deconv4, shortcut3, dropout)
    
    # 32 -> 64
    deconv2 = deconv_block(start_neurons * 2, deconv3, shortcut2, dropout)
    
    # 64 -> 128
    deconv1 = deconv_block(start_neurons, deconv2, shortcut1, dropout)
    
    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(deconv1)
    
    model = Model(input_layer, output_layer)
    return model

In [ ]:
model = build_model(start_neurons=8)
early_stopping = EarlyStopping(patience=50, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               min_delta=1e-4)
model_checkpoint = ModelCheckpoint('weights1.h5', 
                                    save_weights_only=True, 
                                    save_best_only=True)
callbacks = [early_stopping, reduce_lr, model_checkpoint]
optimizer = Adam(lr=0.01)

In [ ]:
model.compile(loss=losses.binary_crossentropy,
              optimizer=Adam(),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, 
          batch_size=32, epochs=200,
          validation_data = [X_val, y_val],
          callbacks = callbacks)

In [ ]:
model.load_weights('weights1.h5')
model.evaluate(X_val, y_val, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(np.argmax(history.history['val_acc']), np.argmax(history.history['acc']), 
               marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.title('model accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim([0.8,1.0])
plt.legend(['train', 'test'], loc= 'upper left')

In [ ]:
data_gen_args = dict(horizontal_flip = True,
                     vertical_flip = True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 2018
bs = 32

image_generator = image_datagen.flow(X_train, 
                                     seed = seed, batch_size = bs, shuffle = True)
mask_generator = image_datagen.flow(y_train, 
                                     seed = seed, batch_size = bs, shuffle = True)

train_generator = zip(image_generator, mask_generator)

In [ ]:
model_checkpoint = ModelCheckpoint('weights2.h5', 
                                    save_weights_only=True, 
                                    save_best_only=True)
callbacks = [early_stopping, reduce_lr, model_checkpoint]

results = model.fit_generator(train_generator, 
                             steps_per_epoch=(len(X_train) // bs),
                             epochs = 200, callbacks = callbacks,
                             validation_data=(X_val, y_val))

In [ ]:
model.load_weights('weights2.h5')
model.evaluate(X_val, y_val, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(np.argmax(history.history['val_acc']), np.argmax(history.history['acc']), 
               marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.title('model accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim([0.8,1.0])
plt.legend(['train', 'test'], loc= 'upper left')

In [ ]:
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_val, verbose = 1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_train > 0.5).astype(np.uint8)

In [ ]:
def plot_sample(X, y, preds, binary_preds, ix= None):
    if ix is None:
        ix = random.randint(0, len(X))
    
    has_mask = y[ix].max() > 0
    
    fig, ax = plt.subplots(1, 4, figsize=(20,20))
    ax[0].imshow(X[ix,...,0], cmap='seismic')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Seismic')
    
    ax[1].imshow(y[ix].squeeze())
    ax[1].set_title('Salt')
    
    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Salt Predicted')
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    
    ax[3].set_title('Salt Predicted Binary')

In [ ]:
plot_sample(X_train, y_train, preds_train, preds_train_t, ix = 100)